In [1]:
import pandas as pd 
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.applications.resnet import ResNet50, ResNet101
from tensorflow.keras.applications import VGG16

#from tensorflow.keras.applications.resnet import ResNet101 as ResNet101
import tensorflow.keras.applications as tfa
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf
#from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers import Adam as Adam
from tensorflow.keras.optimizers import Adagrad

from tensorflow import keras
tf.random.set_seed(42)
import wandb
from wandb.keras import WandbCallback
tf.keras.backend.clear_session()
# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [2]:
dir(tf.keras.applications.resnet)

['ResNet101',
 'ResNet152',
 'ResNet50',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'decode_predictions',
 'preprocess_input']

In [3]:
print(tf.__version__)

2.6.0


In [4]:
meta = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')

In [5]:
def append_ext(fn):
    return fn+".jpg"

In [6]:
labels = meta[['Id','Pawpularity']]
labels['Pawpularity_norm'] = labels['Pawpularity']/100

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
labels_wt_100 = meta[meta['Pawpularity']<100][['Id','Pawpularity']]
labels_wt_100['Pawpularity_norm'] = labels_wt_100['Pawpularity']/100

In [8]:
labels_wt_100["Id"] = labels_wt_100["Id"].apply(append_ext)
labels_wt_100.head(5)

,Id,Pawpularity,Pawpularity_norm
0,0007de18844b0dbbb5e1f607da0606e0.jpg,63,0.63
1,0009c66b9439883ba2750fb825e1d7db.jpg,42,0.42
2,0013fd999caf9a3efe1352ca1b0d937e.jpg,28,0.28
3,0018df346ac9c1d8413cfcc888ca8246.jpg,15,0.15
4,001dc955e10590d3ca4673f034feeef2.jpg,72,0.72


In [9]:
labels.to_csv('labels.csv',encoding='utf-8', index=False)

In [10]:
labels_wt_100.to_csv('labels2.csv',encoding='utf-8', index=False)

It can summarized some of these usage patterns of pre-trained modesl as follows:

**Classifier**: The pre-trained model is used directly to classify new images.

**Standalone Feature Extractor**: The pre-trained model, or some portion of the model, is used to pre-process images and extract relevant features.

**Integrated Feature Extractor**: The pre-trained model, or some portion of the model, is integrated into a new model, but layers of the pre-trained model are frozen during training.

**Weight Initialization**: The pre-trained model, or some portion of the model, is integrated into a new model, and the layers of the pre-trained model are trained in concert with the new model.

It may **not be clear as to which usage of the pre-trained model may yield the best results** on your new computer vision task, therefore some **experimentation may be required**.

In [11]:
def load_train(path, batch_size):
        # path /kaggle/input/petfinder-pawpularity-score/
    labels = pd.read_csv('./labels2.csv')
       
    train_datagen = ImageDataGenerator(validation_split = 0.25, 
                                   rescale = 1/255., 
                                   vertical_flip = True, 
                                   #horizontal_flip = True,
                                   rotation_range = 45,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05)
    
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory= path +'train/',
        x_col='Id',
        y_col='Pawpularity_norm',
        target_size=(160, 160),
        batch_size=batch_size,
        class_mode='raw',
        subset='training',
        seed=42)

    return train_gen_flow

In [12]:
def load_test(path, batch_size):
    labels = pd.read_csv('./labels2.csv')
    test_datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255)
    
# flow_from_dataframe -> Takes the dataframe and the path to a directory + generates batches.
# The generated batches contain augmented/normalized data.
    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe = labels,
        directory = path + 'train/',
        x_col = 'Id',
        y_col = 'Pawpularity_norm',
        target_size = (160, 160),
        batch_size = batch_size,
        class_mode = 'raw',
        subset = 'validation',
        seed = 42)

    return test_gen_flow

In [13]:
# path = '/kaggle/input/petfinder-pawpularity-score/'
# train_gen_flow = load_train(path)

In [14]:
# # getting data from iterator after augmentation
# features, target = next(train_gen_flow)

In [15]:
# # plotting images after augmentation
# fig = plt.figure(figsize=(10,10))
# for i in range(16):
#     fig.add_subplot(4, 4, i+1)
#     plt.imshow(features[i])
#     # to plot compact uncomment _ticks below
#     #plt.xticks([])
#     #plt.yticks([])
#     plt.tight_layout()

In [16]:
# path = '/kaggle/input/petfinder-pawpularity-score/'
# test_gen_flow = load_test(path)

In [17]:
def create_model(input_shape, model_name, optimizer, activation, lr):
    """ 
    Just an example. We get rid of the top (GlobalAveragePooling2D and Dense)
    and reconstract from strach  by adding manually this layers 
    
    backbone (англ. «костяк») — part that left from ResNet
    
    """
    
    backbone = model_name(weights='imagenet', 
                            # we got regression model so we dont define argument classes here
                            # the same for using pre-trained model  as we will get features and not classes
                        input_shape=input_shape,
                            # at the end of ResNet 2 layers given: GlobalAveragePooling2D & Dense. 
                            # if set include_top = False where will be no such layers
                            # set up false, because we dont need Dense layer 
                        include_top=False
                            # we initialiez weights from ResNet that has benn trained on pictures with 1000 classes
                            # if we use this initialisation results should be better than starting from randon initialization
                       )
    
    
    """ 
    We use "Freezing" as the base of our architecture that consist of ResNet. 
    "Freezing" means we will not train it, so the weights will be not changed 
    "Freezing" allows us to get rid of overfitting and increase the speed of NN training:
    gradient descent does not need to calculate derivatives for frozen layers (hurray!). 
    """
    
    backbone.trainable = False

    model = Sequential()
    model.add(backbone)
    
    """
    model.add(GlobalAveragePooling2D())
    model.add(Dense(12, activation='softmax')) 
    """
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation=activation))

    #optim = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer(lr),loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    return model

In [18]:
# image_batch, label_batch = next(iter(train_dataset))
# feature_batch = base_model(image_batch)

In [19]:
# history = model.fit(train_dataset,
#                     epochs=initial_epochs,
#                     validation_data=validation_dataset)

In [20]:
def create_model_2(input_shape, model_name, optimizer, activation, lr):

    backbone = VGG16(weights='imagenet', 
                        input_shape=input_shape,
                        include_top=False)
    
    #backbone.trainable = False
    
    ## Setting the Inputs
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    ## Convolutional Layers
    x = backbone(x, training=False)
    # ----------
    x =  tf.keras.layers.GlobalAveragePooling2D()(x)
    
    ## Flattening the layer
    #x = keras.layers.Flatten()(x)
    ### First Fully Connected layer w/ Dropout
    #x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
    
    # -------------
    
    #x = tf.keras.layers.Dropout(0.2)(x)
    output = keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    ## Returning the model
    #return keras.Model(inputs=inputs, outputs=output)

    #optim = Adam(learning_rate=lr)
    model.compile(optimizer=Adagrad(lr),loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    return model

In [21]:
def train_model(model, train_data, test_data, batch_size = None, epochs = 10,
                steps_per_epoch = None, validation_steps = None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data, 
              validation_data = test_data,
              batch_size = batch_size, 
              epochs = epochs,
              steps_per_epoch = steps_per_epoch,
              validation_steps = validation_steps,
              verbose = 2,
              callbacks=[WandbCallback(
                monitor="val_loss", verbose=0, mode="auto", save_weights_only=(False),
                log_weights=(True), log_gradients=(True), save_model=(True),
                training_data=train_data, validation_data=None, labels=[], data_type=None,
                predictions=0, generator=None, input_type=None, output_type=None,
                log_evaluation=(False), validation_steps=None, class_colors=None,
                log_batch_frequency=None, log_best_prefix="best_", save_graph=(True),
                validation_indexes=None, validation_row_processor=None,
                prediction_row_processor=None, infer_missing_processors=(False),
                log_evaluation_frequency=0)])

    return model

In [22]:
wandb.login(key='620527a80f5b194ce6ba9498879a2ebe65db428d')
# 620527a80f5b194ce6ba9498879a2ebe65db428d

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [23]:
sweep_config = {
  'framework': 'TF',
  'method': 'random', 
  'note':'no_tuning',
  'metric': {
      'name': 'val_root_mean_squared_error',
      'goal': 'minimize'
  },
  'parameters': {
      'batch_size': {
          'values': [32
                    # , 64
                    ]
      },
      'learning_rate':{
          'values': [#0.01, 
                     #0.005, 
                     0.0001 #, 
                     #0.0005, 
                     #0.0001
          ]
      },
      'optimizer': {
        'values': [#'Adam' 
                   'Adagrad'
                   #'SGD'
        ]
        },
#     'fc_layer_size': {
#         'values': [128, 256, 512]
#         },
    'dropout': {
          'values': [0.0
                    # , 0.3, 0.4, 0.5
                    ]
        },
    'input_shape': {
          'values': [(160, 160, 3)]
        },
    'pre_trained_model': {
          'values': [#'ResNet101'
                     #'ResNet50'
                     'VGG16'
                    ]
        },
    'tune_from': {
        'values': ['000']
    },
    'head': {
        'values': [#'GPA_FC_sigmoid'
                    'GPA_sigmoid' #,
                   #'MP_sigmoid'
        ]
#                    'Flatten_FC_sigmoid_DO', 
#                    'Flatten_FC_relu_DO',
#                    'GAP_FC_relu_DO',
#                    'MP_relu128_DO_BN_relu64',
#                    'GAP_relu128_DO_BN_relu64',
#                    'GAP_relu128_DO_BN_relu64_DO',
#                    'GAP_FC_relu_DO_BN',
#                    'GAP_FC_relu512_DO_BN_relu128_DO_BN',
#                    'GAP_relu512_DO_BN_sigmoid',
#                    'MP_DO_FL_relu512']
    },
    'epochs': {
        'values': [10]
    },
    'loss_function': {
        'values': ['mse'
                   #'bce_logits', 
                   #'bce_no_logits'
                  ]
    }
  }
}

In [24]:
sweep_id = wandb.sweep(sweep_config, project="42",entity="42")

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('framework', 'note' were unexpected)


Create sweep with ID: qci4bm2b
Sweep URL: https://wandb.ai/42/42/sweeps/qci4bm2b


In [25]:
def train(config=None):
    
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        print('agent received config: ', config)

        ##loader = build_dataset(config.batch_size)
        # adding our loaders 
    
        ## to do: from config add
        ##        batch size
        ##        pictures augmentation 
        ##        including augmentations functions 
        print('===================')
        print('clearing session...')
        print('===================')
        tf.keras.backend.clear_session()
        
        path = '/kaggle/input/petfinder-pawpularity-score/'
        train_gen_flow = load_train(path, config.batch_size)
        test_gen_flow = load_test(path, config.batch_size)
        
        # build model 
#         pre_trained_model = config['pre_trained_model']
#         fine_tune_from = config['tune_from']
#         head = config['head']
#         activation = config['head']
#         input_shape = config['input_shape']
        activation = 'sigmoid'
        
        model = create_model_2(tuple(config.input_shape), 
                              config.pre_trained_model,
                              config.optimizer,
                              activation,
                               config.learning_rate)
        
#         def create_model_2(input_shape, model_name, optimizer, activation, lr):
        
        # compile model
        
#         lr_with_decay = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=config.learning_rate,
#             decay_steps=100, decay_rate=0.96,
#             staircase=True)
        
        #compiled_model = compile_model(uncompiled_model, config.optimizer, config.loss_function, lr_with_decay)
        #for layer in compiled_model.layers:
        #      print(layer.name, layer.trainable)
        #print('compiled model summary is: ', compiled_model.summary())
        history = train_model(model, train_gen_flow, test_gen_flow, config.batch_size, config.epochs)
    

In [26]:
#wandb.agent(sweep_id, train, count=5)
wandb.agent(
    sweep_id, function=train, entity="42", project="42", count=1
)

wandb: Agent Starting Run: qw4l910i with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	head: GPA_sigmoid
wandb: 	input_shape: [160, 160, 3]
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: mse
wandb: 	optimizer: Adagrad
wandb: 	pre_trained_model: VGG16
wandb: 	tune_from: 000
wandb: Currently logged in as: makcfd (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


agent received config:  {'batch_size': 32, 'dropout': 0, 'epochs': 10, 'head': 'GPA_sigmoid', 'input_shape': [160, 160, 3], 'learning_rate': 0.0001, 'loss_function': 'mse', 'optimizer': 'Adagrad', 'pre_trained_model': 'VGG16', 'tune_from': '000'}
clearing session...



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Found 7218 validated image filenames.
Found 2406 validated image filenames.


2022-02-09 02:15:51.569902: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


...

2022-02-09 02:15:51.713016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 02:15:51.713739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 02:15:51.715552: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-09 02:15:51.717053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Epoch 1/10


2022-02-09 02:15:59.802777: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


226/226 - 249s - loss: 0.0454 - root_mean_squared_error: 0.2132 - val_loss: 0.0479 - val_root_mean_squared_error: 0.2189
Epoch 2/10
226/226 - 158s - loss: 0.0408 - root_mean_squared_error: 0.2020 - val_loss: 0.0439 - val_root_mean_squared_error: 0.2095
Epoch 3/10
226/226 - 159s - loss: 0.0378 - root_mean_squared_error: 0.1945 - val_loss: 0.0416 - val_root_mean_squared_error: 0.2038
Epoch 4/10
226/226 - 158s - loss: 0.0364 - root_mean_squared_error: 0.1907 - val_loss: 0.0400 - val_root_mean_squared_error: 0.1999
Epoch 5/10
226/226 - 158s - loss: 0.0351 - root_mean_squared_error: 0.1874 - val_loss: 0.0388 - val_root_mean_squared_error: 0.1969
Epoch 6/10
226/226 - 157s - loss: 0.0345 - root_mean_squared_error: 0.1858 - val_loss: 0.0378 - val_root_mean_squared_error: 0.1944
Epoch 7/10
226/226 - 157s - loss: 0.0339 - root_mean_squared_error: 0.1842 - val_loss: 0.0371 - val_root_mean_squared_error: 0.1926
Epoch 8/10
226/226 - 158s - loss: 0.0333 - root_mean_squared_error: 0.1826 - val_loss: 

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▂▂▂▁▁▁
root_mean_squared_error,█▆▄▃▂▂▂▁▁▁
val_loss,█▆▄▃▃▂▂▁▁▁
val_root_mean_squared_error,█▆▄▄▃▂▂▁▁▁
best_epoch,9
best_val_loss,0.03579
epoch,9
loss,0.03269
root_mean_squared_error,0.18081
val_loss,0.03579


In [27]:
# input_shape = (160, 160, 3)
# optimizer = Adam
# pre_trained_model = ResNet101
# activation = 'sigmoid'
# lr = 0.001

# model_levels = create_model_2(input_shape, pre_trained_model, optimizer, activation, lr)
# model_to_test = train_model(model_levels, train_gen_flow, test_gen_flow, batch_size = 32)

In [28]:
#model_test.predict()

In [29]:
# input_shape = (150, 150, 3)
# optimizer = Adam
# pre_trained_model = ResNet101
# activation = 'sigmoid'
# lr = 0.001

# model_levels = create_model(input_shape, pre_trained_model, optimizer, activation, lr)
# train_model(model_levels, train_gen_flow, test_gen_flow, batch_size = 32)

# ========================

In [30]:
# input_shape = (150, 150, 3)
# optimizer = Adam
# pre_trained_model = ResNet101
# activation = 'sigmoid'
# lr = 0.001

# model_levels = create_model(input_shape, pre_trained_model, optimizer, activation, lr)
# train_model(model_levels, train_gen_flow, test_gen_flow, batch_size = 32)

In [31]:
# input_shape = (150, 150, 3)
# optimizer = SGD
# pre_trained_model = ResNet101
# activation = 'sigmoid'
# lr = 0.001

# model_levels = create_model(input_shape, pre_trained_model, optimizer, activation, lr)
# train_model(model_levels, train_gen_flow, test_gen_flow, batch_size = 32)

In [32]:
# input_shape = (150, 150, 3)
# optimizer = Adam
# pre_trained_model = InceptionV3
# activation = 'sigmoid'
# lr = 0.001

# model_levels = create_model(input_shape, pre_trained_model, optimizer, activation, lr)
# train_model(model_levels, train_gen_flow, test_gen_flow, batch_size = 32)

In [33]:
# input_shape = (150, 150, 3)
# optimizer = Adam
# pre_trained_model = InceptionV3
# activation = 'tanh'
# lr = 0.001

# model_levels = create_model(input_shape, pre_trained_model, optimizer, activation, lr)
# train_model(model_levels, train_gen_flow, test_gen_flow, batch_size = 32)

Initial run 
Changes 1: backbone is not trainable. Results: 21.1810

Changes 2: ResNet101, sigmoid, normalized target, Adam, lr = 0.005. Results: 0.2118

Changes 3: TBD. Results:

Changes 4: TBD. Results:

Changes 5: TBD. Results:


